In [1]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import random #GGC added library

# Fixed parameters (number of samples)

In [2]:
#Number of samples determined by Chebyshev inequality
samples = 10000

# Variable parameters (qubits and layers)

In [3]:
#List of number of qubits
#qubits = [4]
qubits = [8]

#List of number of layers
#layers = [1, 2, 3, 4, 5]
#layers = [int(20/11), int(30/11), int(40/11), int(60/11), int(70/11), int(80/11), int(90/11), int(100/11), int(110/11)] #/x porque agora temos x gates por layer no no-connection
layers = [int(30/23), int(60/23), int(90/23), int(120/23), int(180/23), int(210/23), int(240/23)]

# Function to calculate Meyer-Wallach mean and standard deviation

In [4]:
def ent_cap(samples, circuit, nlayers):
    MW_mean = 0
    MW_mean_sqr = 0
    
    for i in range(samples):
        pur_sum = 0
        for k in range(wires):
            state = circuit(k)
            pur_sum = pur_sum + np.trace(np.matmul(state, state))

        MW_measure = 2*(1-1/wires*pur_sum)
        MW_measure_sqr = MW_measure**2

        MW_mean = MW_mean + MW_measure
        MW_mean_sqr = MW_mean_sqr + MW_measure_sqr
        
    MW_mean = MW_mean/samples
    MW_mean_sqr = MW_mean_sqr/samples
    MW_standev = np.sqrt(MW_mean_sqr - MW_mean**2)
    return(MW_mean, MW_standev)

# Code that executes the different conditions

In [5]:
now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")
print("Tempo inicial: ")
print(now0)
print()

for wires in qubits:
    dev = qml.device("default.qubit", wires=wires, shots=None)    
    
    for nlayers in layers:
        
        #GGC addition of random G3 circuit (I have left the input parameter (though useless) in order to keep code shape

#============================================================================================================================================

        def layer_random():
            for index in range(8):
                i1 = random.sample(range(0,wires), 2)
    
                i2 = random.randint(1,2)
                if   i2 == 1 :
                        qml.Hadamard(wires=[i1[0]])
                elif i2 == 2 :
                        qml.S(wires=[i1[0]]) #Other option: qml.S(wires=[i1[0]])
                    
            #CNOTS(Diferentes topologias)

            #qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain") #linear

            #qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain") #ring
            #qml.CNOT(wires=[wires-1,0])

            for i in range(wires-1): #star
                qml.CNOT(wires = [0, i+1])

#============================================================================================================================================
        def randomC(k):
            qml.layer(layer_random, nlayers)
            return qml.density_matrix([k])
        
        
        random_circuit = qml.QNode(randomC, dev)
        
        
        ent_random, ent_random_standev = ent_cap(samples, random_circuit, nlayers)
        
        f = open("entanglement_G2star_qubits{0}_layers{1}_samples{2}.txt"
                         .format(wires,nlayers,samples), "w")
        
        print('Circuit,MW_mean,MW_standdev', file = f)
        print('Random Circuit G3,',ent_random,',', ent_random_standev, file = f)
        
        print('Execution: qubits=',wires,'; layers=', nlayers)
        f.close()
        now1 = dt.datetime.now() #monitorar tempo de início
        now1 = now1.strftime("%Y-%m-%d %H:%M:%S")
        print("Tempo: ",now1)
        print()
        
now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")

f.close()

print()
print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Tempo inicial: 
2025-07-08 00:37:11

Execution: qubits= 8 ; layers= 1
Tempo:  2025-07-08 00:41:29

Execution: qubits= 8 ; layers= 2
Tempo:  2025-07-08 00:50:22

Execution: qubits= 8 ; layers= 3
Tempo:  2025-07-08 01:02:24

Execution: qubits= 8 ; layers= 5
Tempo:  2025-07-08 01:16:10

Execution: qubits= 8 ; layers= 7
Tempo:  2025-07-08 01:32:04

Execution: qubits= 8 ; layers= 9
Tempo:  2025-07-08 01:51:54

Execution: qubits= 8 ; layers= 10
Tempo:  2025-07-08 02:13:07


Tempo inicial: 
2025-07-08 00:37:11
Tempo final:
2025-07-08 02:13:07
